# Select-Swap QROM

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `SelectSwapQROM`
Gate to load `data[l]` in the target register when the selection register stores integer `l`.

Let `N` be the number of data elements to load; and `b` be the bit-size of the target
register in which data elements should be loaded.

The `SelectSwapQROM` is a hybrid of the following two existing primitives:

 - Unary Iteration based `QROM` requires O(N) T-gates to load `N` data
   elements into a `b`-sized target register. Note that the T-complexity is independent of `b`.
 - `SwapWithZeroGate` can swap a `b`-sized register at index `x` with a `b`-sized
   register at index `0` using O(b) T-gates, if the selection register stores integer `x`.
   Note that the swap complexity is independent of the iteration length `N`.

The `SelectSwapQROM` uses a square root decomposition by combining the above two approaches to
further optimize the T-gate complexity of loading `N` data elements, each into a `b`-sized
target register as follows:

 - Divide the `N` data elements into batches of size `B` (a variable) and
   load each batch simultaneously into `B` distinct target signature using the conventional
   QROM. This has T-complexity `O(N / B)`.
 - Use `SwapWithZeroGate` to swap the `i % B`'th target register in batch number `i / B`
   to load `data[i]` in the 0'th target register. This has T-complexity `O(B * b)`.

Thus, the final T-complexity of `SelectSwapQROM` is $O(B * b + N / B)$ T-gates; where $B$ is
the block-size with an optimal value of $O(\sqrt(N / b))$.

This improvement in T-complexity is achieved at the cost of using an additional $O(B * b)$
ancilla qubits, with a nice property that these additional ancillas can be "dirty".
They don't need to start in the |0> state and can be borrowed from other parts of the
algorithm. The these dirty ancillas are restored to their original state after the operation.

#### Parameters
 - `data`: Sequence of integers to load in the target register. If more than one sequence is provided, each sequence must be of the same length.
 - `target_bitsizes`: Sequence of integers describing the size of target register for each data sequence to load. Defaults to `max(data[i]).bit_length()` for each i.
 - `block_size`: Load batches of `B` data elements in each iteration of traditional QROM (N/B iterations required). Complexity of SelectSwap QROAM scales as `O(B * b + N / B)`, where `B` is the block_size. Defaults to optimal value of `O(sqrt{N / b})`. 

#### Registers
 - `selection`: The selection register of size `logN` to index into each `data`.
 - `target{i}_`: The target registers, one for each `data` to load the data. 

#### References
[Trading T-gates for dirty qubits in state preparation and unitary synthesis](https://arxiv.org/abs/1812.00954).
    Low, Kliuchnikov, Schaeffer. 2018.


In [ ]:
from qualtran.bloqs.select_swap_qrom import SelectSwapQROM

### Example Instances

In [ ]:
rs = np.random.RandomState(52)
datas = rs.randint(0, 256, size=(3, 11))
ss_qrom = SelectSwapQROM(*datas)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([ss_qrom],
           ['`ss_qrom`'])

### Call Graph

In [ ]:
ss_qrom_g, ss_qrom_sigma = ss_qrom.call_graph()
show_call_graph(ss_qrom_g)
show_counts_sigma(ss_qrom_sigma)